## Convolutional Neural Network

In this notebook you will learn to distinguish dogs from cats!

Data:
https://drive.google.com/drive/folders/1nzVk4GOvKR6P87uPszUkKMPtaXV_wrZf?usp=sharing

Fill all the necessary gaps in cells below and fit neural networks for solving the binary classification task.

## Task 1:

1. Build and fit CNN with 3 convolutional layers for binary classification
2. Evaluate accuracy on test data
3. Plot the graphs for Loss(number_of_epochs) and Accuracy(number_of_epochs)

First, let's load all the necessary functions:

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]= "-1"

In [2]:
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense, Conv2D, MaxPooling2D
from keras.applications import VGG16, ResNet101V2
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

import matplotlib.image as mpimg
import matplotlib.pyplot as plt

import cv2
import numpy as np

The images collected for training and testing the deep learning model must be prepared: split the entire set into a training, validation and test sample, observing the balancing of classes (with binary classification they should be approximately equal in all three samples).

This has _already_ been done: in the Cats_and_Dogs directory there are three subdirectories: train, test and val - training, test and validation samples, respectively.

In [3]:
# Initialize the folders with train, test and validation datasets (in "/My Drive/..." or from your local repository where you have downloaded data):

train = "data/train/"
val =   "data/val/"
test =  "data/test/"

# The shape of the RGB image
img_width, img_height, channels = 150, 150, 3 # you can try different sizes

# input shape
input_shape = (img_width, img_height, 3)
# position matters!
# Number_of_channels can be at the first or the last position
# in our case - "channels last"

# minibatch size
batch_size = 64
# train set size
nb_train_samples = 20000
# validation set size 
nb_validation_samples = 2500
# test set size
nb_test_samples = 2500

## Prepare the data.

You don’t have to manually change the shapes of 25000 images and convert them into the necessary format for keras (img_width, img_height, 3).

We will use the built-in image preprocessing function _ImageGenerator()_.

It performs scaling, resizes selected images and prepares batches (mini-samples) to train the model.

In [4]:
datagen = ImageDataGenerator(rescale=1./255)

train_generator = datagen.flow_from_directory(
    train,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode="binary"
)

val_generator = datagen.flow_from_directory(
    val,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode="binary"
)

test_generator = datagen.flow_from_directory(
    test,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode="binary"
)

Found 20000 images belonging to 2 classes.
Found 2490 images belonging to 2 classes.
Found 2500 images belonging to 2 classes.


Set the network architecture by sequentially adding layers to it:
1. A convolutional layer with 16 neurons, filter size 3x3. Activation function - 'relu'
2. MaxPooling layer with filter size 2x2.
3. A convolutional layer with 32 neurons, filter size 3x3. Activation function - 'relu'
4. MaxPooling layer with filter size 2x2.
5. A convolutional layer with 64 neurons, filter size 3x3. Activation function - 'relu'
6. MaxPooling layer with filter size 2x2.
7. Operation model.add (Flatten ()), which makes a one-dimensional vector of the resulting feature maps.
8. A fully connected layer with 64 neurons. Activation function - 'relu'
9. Use model.add (Dropout (0.5)) which excludes the edge from the current layer in the computational graph with a 50% probability to avoid overfitting.
10. A fully connected layer with 1 neuron. Activation function - 'sigmoid', because binary classification model.

Add to the model all the missing layers, by analogy with the already specified.
Keras documentation: https://keras.io/layers/about-keras-layers/

In [5]:
model = Sequential()

# 1: +Convolutional
model.add(Conv2D(16, (3, 3), input_shape=(150, 150, 3)))
model.add(Activation("relu"))
# 2: +Pooling
model.add(MaxPooling2D(pool_size=(2, 2)))
# 3:
model.add(Conv2D(32, (3, 3)))
model.add(Activation("relu"))
# 4:  +Pooling
model.add(MaxPooling2D(pool_size=(2, 2)))
# 5:  +Convolutional
model.add(Conv2D(32, (3, 3)))
model.add(Activation("relu"))
#     +Relu
model.add(Activation("relu"))
# 6:  +Pooling
model.add(MaxPooling2D(pool_size=(2, 2)))
# 7:  +Flattening
model.add(Flatten())
# 8:  +Convolutional
model.add(Dense(64, activation="relu"))
# 9:  +Dropout
model.add(Dropout(0.2))
# 10: +Dense
model.add(Dense(1,activation="sigmoid"))
#     +Sigmoid

In [6]:
model.compile(
    loss="binary_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
)

In [7]:
model_history = model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=15, #try different number of epochs: 10, 15, 20; check the loss and accuracy;
    validation_data=val_generator,
    validation_steps=nb_validation_samples // batch_size
)

c:\users\sultan\documents\work\master_1\ml_itmo\venv\lib\site-packages\tensorflow\python\keras\engine\training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/15
312/312 [==============================] - 244s 780ms/step - loss: 0.6518 - accuracy: 0.6089 - val_loss: 0.5160 - val_accuracy: 0.7470

KeyboardInterrupt: 

In [ ]:
scores = model.evaluate_generator(test_generator, nb_test_samples // batch_size)
print(f"Accuracy on test data: {scores[1] * 100 :.2f}%")

Plot the graphs:
- Loss(Number of epochs)
- Accuracy(Number of epochs)

In [ ]:
print(model_history.history.keys())

In [ ]:
plt.figure(figsize=(12, 4))

# Loss(Number of epochs)
plt.subplot(1, 2, 1)
plt.plot(model_history.history["loss"])
plt.plot(model_history.history["val_loss"])
plt.title("Loss")
plt.ylabel("Loss")
plt.xlabel("Epoch")
plt.legend(["train", "test"], loc="upper left")

# Accuracy(Number of epochs)
plt.subplot(1, 2, 2)
plt.plot(model_history.history["accuracy"])
plt.plot(model_history.history["val_accuracy"])
plt.title("Accuracy")
plt.ylabel("Accuracy")
plt.xlabel("epoch")
plt.legend(["train", "test"], loc="upper left")

plt.show()

In [ ]:
img = mpimg.imread("data/Deeper.jpeg")
plt.figure(figsize = (10, 20))
plt.imshow(img)
plt.show()

Let's try to improve the quality of recognition, using the method of transfer lerning. 

We will use weights of deep neural networks already trained on large dataset such as  ImageNet, and provide fine tuning of several additional dense layers on new data relevant to the current classification task. The more new images will differ from those on which the network has been trained, the more layers will need to be “retrained” in order to get good classification accuracy. The intuition here is that the model has already learned how to highlight the necessary features on the images in the large dataset, it only needs to be “tweaked” for a specific task.

## Task 2

1. Build and fit Transfer Learning model using pre-trained VGG16-model weights from keras application.
2. Do the same with **another avaliable pre-trained deep learning model** from keras application https://keras.io/api/applications/.
2. Evaluate accuracy on test data for p.1 and p.2
3. Plot the graphs for Loss(number_of_epochs) and Accuracy(number_of_epochs)
4. Check the performance of your model with the custom image of cat or dog (so the model will tell which class this image belongs to). Develop the function for the inference of the best algorithm.

In [ ]:
# First, download the weights of the VGG16 network trained on the ImageNet dataset:
vgg16_net = VGG16(
    weights="imagenet",
    include_top=False, # we take only part of the "convolution" and add the last layers
    input_shape=(150, 150, 3)
)
vgg16_net.trainable = False # don't retrain the network
vgg16_net.summary()

We construct our model of "transfer learning" by adding two fully connected layers to VGG16

In [ ]:
# add layers to VGG16:
model = Sequential()
model.add(vgg16_net)
# + flattening
model.add(Flatten())
# + dense connected layer with 256 neurons
# + ReLu
model.add(Dense(256, activation="relu"))
# + Dropout
model.add(Dropout(0.5))
# + full layer with 1 neuron
# + sigmoid
model.add(Dense(1,activation="sigmoid"))
model.summary()

In [ ]:
model.compile(
    loss="binary_crossentropy",
    optimizer=Adam(lr=1e-5),
    metrics=["accuracy"]
)

E.g., it was like:

In [ ]:
img = mpimg.imread("data/VGG16.png")
plt.figure(figsize = (10, 20))
plt.imshow(img)
plt.show()

**and** it becomes:

In [ ]:
img = mpimg.imread("data/VGG162.png")
plt.figure(figsize = (10, 20))
plt.imshow(img)
plt.show()

In [ ]:
# We also use the generator to train the model (similar to the fit method)
# Without using a GPU, learning 1 epoch of such a network will take about an hour. Plan your time =)
# If you have access to a GPU, you can try 10-12 epochs - the quality should increase even more.

model_history_vgg = model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=5,
    validation_data=val_generator,
    validation_steps=nb_validation_samples // batch_size
)

In [ ]:
scores = model.evaluate_generator(test_generator, nb_test_samples // batch_size)
print(f"Accuracy on test data: {scores[1] * 100 :.2f}%")

In [ ]:
plt.figure(figsize=(12, 4))

# Loss(Number of epochs)
plt.subplot(1, 2, 1)
plt.plot(model_history_vgg.history["loss"])
plt.plot(model_history_vgg.history["val_loss"])
plt.title("Loss")
plt.ylabel("Loss")
plt.xlabel("epoch")
plt.legend(["train", "test"], loc="upper left")

# Accuracy(Number of epochs)
plt.subplot(1, 2, 2)
plt.plot(model_history_vgg.history["accuracy"])
plt.plot(model_history_vgg.history["val_accuracy"])
plt.title("Accuracy")
plt.ylabel("Accuracy")
plt.xlabel("epoch")
plt.legend(["train", "test"], loc="upper left")

plt.show()

In [ ]:
model.save("data/cats_and_dogs_vgg16.h5")

In [ ]:
def predict_class(custom_img_path, model_animal):
    test_img = cv2.imread(custom_img_path)
    test_img = cv2.resize(test_img, (150, 150))
    test_img = np.array(test_img).reshape((1, 150, 150, 3))

    result = model_animal.predict(test_img)
    if result.astype(int).item() == 0:
        print("Cat")
    elif result.astype(int).item() == 1:
        print("Dog")
    else:
        print("Don't know")
        

In [ ]:
cat_img = "data/cat.jpg"
img = mpimg.imread(cat_img)
plt.figure(figsize = (10, 10))
plt.imshow(img)
plt.show()

In [ ]:
predict_class(cat_img, model)

In [ ]:
dog_img = "data/dog.jpg"
img = mpimg.imread(dog_img)
plt.figure(figsize = (10, 10))
plt.imshow(img)
plt.show()

In [ ]:
predict_class(dog_img, model)

In [ ]:
resnet = ResNet101V2(
    weights="imagenet", 
    include_top=False, 
    input_shape=(150, 150, 3)
)
resnet.trainable = False               
resnet.summary()

In [ ]:
# add layers to ResNet101V2:
model = Sequential()
model.add(resnet)
# + flattening
model.add(Flatten())
# + dense connected layer with 256 neurons
# + ReLu
model.add(Dense(256, activation="relu"))
# + Dropout
model.add(Dropout(0.5))
# + full layer with 1 neuron
# + sigmoid
model.add(Dense(1,activation="sigmoid"))
model.summary()

In [ ]:
model.compile(
    loss="binary_crossentropy",
    optimizer=Adam(lr=1e-5), 
    metrics=["accuracy"]
)

In [ ]:
model_histiry_resnet = model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=5,
    validation_data=val_generator,
    validation_steps=nb_validation_samples // batch_size
)

In [ ]:
model.save('cats_and_dogs_ResNet101V2.h5')

In [ ]:
scores = model.evaluate_generator(test_generator, nb_test_samples // batch_size)
print(f"Accuracy on test data: {scores[1] * 100 :.2f}%")

In [ ]:
plt.figure(figsize=(12, 4))

# Loss(Number of epochs)
plt.subplot(1, 2, 1)
plt.plot(model_histiry_resnet.history["loss"])
plt.plot(model_histiry_resnet.history["val_loss"])
plt.title("Loss")
plt.ylabel("loss")
plt.xlabel("epoch")
plt.legend(["train", "test"], loc="upper left")

# Accuracy(Number of epochs)
plt.subplot(1, 2, 2)
plt.plot(model_histiry_resnet.history["accuracy"])
plt.plot(model_histiry_resnet.history["val_accuracy"])
plt.title("model accuracy")
plt.ylabel("accuracy")
plt.xlabel("epoch")
plt.legend(["train", "test"], loc="upper left")

plt.show()

In [ ]:
predict_class(cat_img, model)

In [ ]:
predict_class(dog_img, model)